In [3]:
suppressMessages(library('DESeq2'))
library('BiocParallel')
register(MulticoreParam(4))

In [4]:
# load data
countData <- read.csv(file = "/fast/scratch/users/peidlis_c/countData.csv", row.names=1)
colData <- read.csv(file = "/fast/scratch/users/peidlis_c/colData.csv")

In [5]:
colData

patient,celltype,disease,ncells
<chr>,<chr>,<chr>,<int>
BAL009,Ciliated,COVID19,233
BAL009,Secretory,COVID19,23
BAL009,Basal,COVID19,29
BAL009,Monocyte,COVID19,141
BAL009,Md_macrophage,COVID19,271
BAL009,Alveolar_macrophage,COVID19,12
BAL009,Neutrophil,COVID19,407
BAL009,CD4_Tcell,COVID19,18
BAL009,CD8_Tcell,COVID19,15


In [6]:
celltype='Neutrophil'

In [7]:
# I could encode time as a numerical variable though... but I bet deseq will throw an error there.
# select times where we can compare organisms (otherwise this will be statistically difficult / impossible).
mask = colData$celltype %in% c(celltype)
scolData = colData[mask, ]
scountData = countData[, mask]
# head(scolData)

In [8]:
# this is where the magic happens
design = ~ disease
reduced = ~ 1

# Setup data and supply design matrix
dds <- DESeqDataSetFromMatrix(countData = scountData, colData = scolData,
                              design = design)

converting counts to integer mode

Warning message in DESeqDataSet(se, design = design, ignoreRank):
“some variables in design formula are characters, converting to factors”


In [9]:
# Filter genes below 10 counts in total
dds <- dds[rowSums(counts(dds)) >= 10,]
# Setup deseq with single cell recommendations, add reduced design matrix
dds <- DESeq(dds, test="LRT", minReplicatesForReplace=Inf, reduced=reduced)

estimating size factors

estimating dispersions

gene-wise dispersion estimates

mean-dispersion relationship

final dispersion estimates

fitting model and testing



In [10]:
dds

class: DESeqDataSet 
dim: 9251 24 
metadata(1): version
assays(4): counts mu H cooks
rownames(9251): AL669831.5 LINC00115 ... MT-ND6 MT-CYB
rowData names(21): baseMean baseVar ... deviance maxCooks
colnames(24): X6 X18 ... X284 X319
colData names(5): patient celltype disease ncells sizeFactor

In [11]:
# Run deseq2
res <- results(dds)
resOrdered <- res[order(res$pvalue),]
resOrdered

log2 fold change (MLE): disease pneumonia vs COVID19 
LRT p-value: '~ disease' vs '~ 1' 
DataFrame with 9251 rows and 6 columns
            baseMean log2FoldChange     lfcSE      stat      pvalue        padj
           <numeric>      <numeric> <numeric> <numeric>   <numeric>   <numeric>
C1QA         9.44533        3.54390  0.676555   33.5696 6.87598e-09 3.66696e-05
MYOF         3.48569        3.69502  0.788228   28.1778 1.10667e-07 2.76962e-04
C1QB        16.96813        3.57188  0.763493   27.5160 1.55801e-07 2.76962e-04
MT2A        67.79880        2.63783  0.608455   24.2877 8.29664e-07 1.10615e-03
C1QC        19.80480        3.51417  0.861392   21.0879 4.38700e-06 4.67917e-03
...              ...            ...       ...       ...         ...         ...
CCT8        2.362235       0.677385  1.038569  0.548424          NA          NA
MAP3K7CL    0.490662       2.058329  3.630072  0.493376          NA          NA
TMEM50B     0.930682       0.850607  0.896054  2.152258          NA     

In [12]:
write.csv(as.data.frame(resOrdered), file=paste0("./deseq2_results_", celltype, "_WautersLambrecht210121_pneumonia_vs_COVID19.csv"))